In [ ]:
import tensorflow as tf
import random
# import matplotlib.pyplot as plt
#tf.set_random_seed(777)  # for reproducibility

from tensorflow.examples.tutorials.mnist import input_data
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

nb_classes = 10
numb_of_neurons=512
learning_rate=0.001

X = tf.placeholder(tf.float32, [None, 784])# MNIST data image of shape 28 * 28 = 784
Y = tf.placeholder(tf.float32, [None, nb_classes])# 0 - 9 digits recognition = 10 classes
keep_prob = tf.placeholder(tf.float32)

W1 = tf.get_variable("W1", shape=[784, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([numb_of_neurons]))
W2 = tf.get_variable("W2", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([numb_of_neurons]))
W3 = tf.get_variable("W3", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([numb_of_neurons]))
W4 = tf.get_variable("W4", shape=[numb_of_neurons, numb_of_neurons],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([numb_of_neurons]))
W5 = tf.get_variable("W5", shape=[numb_of_neurons, nb_classes],initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([nb_classes]))

with tf.name_scope("layer1") as scope:
    # Hypothesis (using Relu)
    L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)
    
with tf.name_scope("layer2") as scope:
    # Hypothesis (using Relu)
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

with tf.name_scope("layer3") as scope:
    # Hypothesis (using Relu)
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
    
with tf.name_scope("layer4") as scope:
    # Hypothesis (using Relu)
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

with tf.name_scope("layer5") as scope:
    # Hypothesis (using softmax)
    #hypothesis = tf.nn.softmax(tf.matmul(layer2, W3) + b3)
    hypothesis = tf.matmul(L4, W5) + b5
    

with tf.name_scope("cost") as scope:
    #cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, l   abels=Y))
    cost_summ=tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#     optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05).minimize(cost)
    #GradientDescent doesn't work well.. I don't why Adam is better than Gradient.

is_correct= tf.equal(tf.arg_max(hypothesis,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(is_correct,tf.float32))

training_epochs=15
batch_size=500

with tf.Session() as sess:    
    merged_summary= tf.summary.merge_all()
    #step3 create wirter after creting session
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter('/home/jeongjaepark/HistNet/logs')
    writer.add_graph(sess.graph)
    step=0
    for epoch in range(training_epochs):
        avg_cost=0
        total_batch=int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            feed_dict={X: batch_xs, Y:batch_ys,keep_prob:0.7}
            c, _ = sess.run([cost, optimizer],feed_dict=feed_dict)
            avg_cost += c / total_batch
            
            s=sess.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s,global_step=step)
            step+=1
        
        print("Epoch = ", '%4d'%(epoch+1),'cost = ','{:.9f}'.format(avg_cost))
    # Test the model using test sets
    
    r = random.randint(0, mnist.test.num_examples - 1)
    print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1}))        
    print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
    print("Prediction: ", sess.run(tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1],keep_prob:1}))

In [ ]:
# Lab 7 Learning rate and Evaluation
import tensorflow as tf
import random
# import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.random_normal([10]))

hypothesis = tf.matmul(X, W) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))


In [ ]:
# Lab 10 MNIST and NN
import tensorflow as tf
import random
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

# weights & bias for nn layers
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist.test.images[r:r + 1]}))



In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io, filters
from mlxtend.preprocessing import one_hot
import random

# set train data path
train_list_path = 'data/images_path/train.txt'
train_data_path = 'data/images/train/%s'


class DataRead:
    def __init__(self, data_list_path, data_path, batch_size, nb_classes):
        self.data_list_path = data_list_path
        self.data_path = data_path
        self.batch_num = 0
        self.batch_size = batch_size
        self.nb_classes = nb_classes

        with open(self.data_list_path) as f:
            self.data_list = f.readlines()
        self.total_batch = int(np.floor(np.size(self.data_list) / self.batch_size))

        self.label_list = [int(d.split()[1]) for d in self.data_list]
        self.data_list = [train_data_path % d.split()[0] for d in self.data_list]

        # shuffle data
        ind_shuff = np.arange(np.size(self.data_list))
        random.shuffle(ind_shuff)
        self.data_list = [self.data_list[i] for i in ind_shuff]
        self.label_list = [self.label_list[i] for i in ind_shuff]

    def Next_Batch(self):
        batch_num = int(self.batch_num)
        batch_size = int(self.batch_size)

        data_coll = io.ImageCollection(
            self.data_list[batch_num * batch_size:(batch_num + 1) * batch_size])
        data = np.array([d for d in data_coll])

        label_coll = self.label_list[batch_num * batch_size:(batch_num + 1) * batch_size]
        label = one_hot(label_coll, self.nb_classes)

        self.batch_num = self.batch_num + 1
        if self.batch_num > self.total_batch:
            self.batch_num = 0
        return (data / 255.0, label)

    def All_Data(self):
        data_coll = io.ImageCollection(
            self.data_list)
        data = np.array([d for d in data_coll])

        label_coll = self.label_list
        label = one_hot(label_coll, self.nb_classes)

        return (data / 255.0, label)


nb_classes = 13
learning_rate = 0.001
training_epochs = 45
batch_size = 256
train = DataRead(train_list_path, train_data_path, batch_size=batch_size, nb_classes=13)

keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 256, 256, 3])
X_img = X
Y = tf.placeholder(tf.float32, [None, nb_classes])

W1 = tf.Variable(tf.random_normal([11, 11, 3, 96], stddev=0.001))
W2 = tf.Variable(tf.random_normal([5, 5, 96, 256], stddev=0.001))
W3 = tf.Variable(tf.random_normal([3, 3, 256, 384], stddev=0.001))
W4 = tf.Variable(tf.random_normal([3, 3, 384, 384], stddev=0.001))
W5 = tf.Variable(tf.random_normal([3, 3, 384, 256], stddev=0.001))
W6 = tf.get_variable("W6", shape=[8 * 8 * 256, 4096], initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.Variable(tf.random_normal([4096]))
W7 = tf.get_variable("W7", shape=[4096, 1000], initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.Variable(tf.random_normal([1000]))
W8 = tf.get_variable("W8", shape=[1000, 100], initializer=tf.contrib.layers.xavier_initializer())
b8 = tf.Variable(tf.random_normal([100]))
W9 = tf.get_variable("W9", shape=[100, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
b9 = tf.Variable(tf.random_normal([nb_classes]))

with tf.name_scope("layer1") as scope:
    L1 = tf.nn.conv2d(X_img, W1, strides=[1, 4, 4, 1], padding='SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

with tf.name_scope("layer2") as scope:
    L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

with tf.name_scope("layer3") as scope:
    L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

with tf.name_scope("layer4") as scope:
    L4 = tf.nn.conv2d(L3, W4, strides=[1, 1, 1, 1], padding='SAME')
    L4 = tf.nn.relu(L4)
    L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

with tf.name_scope("layer5") as scope:
    L5 = tf.nn.conv2d(L4, W5, strides=[1, 1, 1, 1], padding='SAME')
    L5 = tf.nn.relu(L5)
    L5 = tf.nn.max_pool(L5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    L5 = tf.reshape(L5, [-1, 8 * 8 * 256])
    L5 = tf.nn.dropout(L5, keep_prob=keep_prob)

with tf.name_scope("layer6") as scope:
    L6 = tf.nn.relu(tf.matmul(L5, W6) + b6)
    L6 = tf.nn.dropout(L6, keep_prob=keep_prob)

with tf.name_scope("layer7") as scope:
    L7 = tf.nn.relu(tf.matmul(L6, W7) + b7)
    L7 = tf.nn.dropout(L7, keep_prob=keep_prob)

with tf.name_scope("layer8") as scope:
    L8 = tf.nn.relu(tf.matmul(L7, W8) + b8)
    L8 = tf.nn.dropout(L8, keep_prob=keep_prob)

with tf.name_scope("layer9") as scope:
    logits = tf.matmul(L8, W9) + b9

with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    cost_summ=tf.summary.scalar("cost",cost)
    
with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

with tf.name_scope("prediction") as scope:
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_summ=tf.summary.scalar("accuracy",accuracy)


# train my model
with tf.Session() as sess:
    print('Learning started. It takes sometime.')
    merged_summary = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('./logs/KIST/learning_rate_0_001')
    writer.add_graph(sess.graph)
    step = 0
    for epoch in range(training_epochs):
        avg_cost = 0
        avg_a=0
        total_batch = train.total_batch

        for i in range(total_batch):
            batch_xs, batch_ys = train.Next_Batch()
            feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
            c, _ ,a= sess.run([cost, optimizer,accuracy], feed_dict=feed_dict)
            avg_cost += c / total_batch
            avg_a+=a / total_batch
            
            s=sess.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, global_step=step)
            step += 1

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost),'accuracy =', '{:.9f}'.format(avg_a))

    val_list_path = 'data/images_path/val.txt'
    val_data_path = 'data/images/val/%s'
    val = DataRead(val_list_path, val_data_path, batch_size=100, nb_classes=13)

    val_x, val_y = val.All_Data()
    print('Accuracy:', sess.run(accuracy, feed_dict={X: val_x, Y: val_y, keep_prob: 1}))


In [ ]:
import tensorflow as tf
import numpy as np
from skimage import data, io, filters
from mlxtend.preprocessing import one_hot
import matplotlib.pyplot as plt
import random
import glob,os


dataset_path="/home/jeongjaepark/HistNet/data/Maps1"

#read data_list
os.chdir(dataset_path)
files=[file for file in glob.glob("*.txt")]
NumofTest=int(len(files)*0.2)
data_path=[dataset_path+'/'+f for f in files]

class DataRead:
    def __init__(self, data_path,batch_size,nb_classes,k_fold=10):
        self.data_path=data_path
        self.batch_size = batch_size
        self.nb_classes=nb_classes
        first=1
        
        #data read & preprocessing
        for j,file_list in enumerate(data_path):
            with open(file_list) as f:
                tmp=f.readlines()
                f.close()

            #data preprocessing
            data_string=[d.split(",")[:] for d in tmp]
            data_len=np.size(data_string,0)
            for i in range(data_len):
                data_string[i][-1]=float(data_string[i][-1]=='t\n')
            tmp=np.array([[float(d) for d in line ]for line in data_string])
            
            #shuffle
            ind_shuff=np.arange(data_len)
            random.shuffle(ind_shuff)
            
            if(first):
                data=tmp[ind_shuff,:]
                count=[data_len]
                data_ind=[0]
                first=0
            else:
                data=np.append(data,tmp[ind_shuff,:],axis=0)
                count=np.append(count,data_len)
                data_ind=np.append(data_ind,data_ind[j-1]+count[j-1])             
                
        
        self.data=data
        self.count=count
        self.data_ind=data_ind
        
    def data_select(self,data_number):
        ind=np.ones(len(self.count), dtype=bool)
        ind[data_number]=False
        train_ind_s=self.data_ind[ind]
        train_ind_n=self.count[ind]
        
        test_ind_s=self.data_ind[data_number]
        test_ind_n=self.count[data_number]
        
        self.train_ind=[]
        for i,start in enumerate(train_ind_s):
            self.train_ind.extend([j for j in range(start,start+train_ind_n[i])])
            
        self.test_ind=[]
        for i,start in enumerate(test_ind_s):
            self.test_ind.extend([j for j in range(start,start+test_ind_n[i])])
        
        self.train=DataRead.distribution(self.data[self.train_ind,:].copy(),self.batch_size,self.nb_classes)
        self.test=DataRead.distribution(self.data[self.test_ind,:].copy(),self.batch_size,self.nb_classes)
    
    class distribution:
        def __init__(self,data,batch_size,nb_classes):
            self.batch_size=int(batch_size)
            self.batch_num=0
            self.data_len=np.size(data,0)
            
            ind_shuff=np.arange(self.data_len)
            random.shuffle(ind_shuff)
            data=data[ind_shuff,:]
            self.X=data[:,:-1]
            Y=[int(t) for t in data[:,-1]]#.reshape(-1,1)
            self.Y=one_hot(Y, nb_classes)
            self.total_batch=int(np.floor(self.data_len/ self.batch_size))
            
        def Next_Batch(self):
            batch_num = int(self.batch_num)
            batch_size = int(self.batch_size)

            data=self.X[batch_num * batch_size:(batch_num + 1) * batch_size,:]
            label=self.Y[batch_num * batch_size:(batch_num + 1) * batch_size,:]
            self.batch_num = self.batch_num + 1
            if self.batch_num > self.total_batch:
                self.batch_num = 0
            return (data , label)

            
#get possible combinations from data_list
data_list=[]
for i in range(len(files)):
    for j in np.arange(i+1,len(files)):
        data_list.append([i,j])
data_list=np.array(data_list)


nb_classes = 2
learning_rate = 0.01
training_epochs = 45
batch_size = 46
data=DataRead(data_path,batch_size,nb_classes)


In [ ]:


keep_prob = tf.placeholder(tf.float32)


X=tf.placeholder(tf.float32,[None, 1440])
X_img=tf.reshape(X,[-1,45,32,1])
Y=tf.placeholder(tf.float32,[None,nb_classes])

W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.001))
W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.001))
W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.001))
W4 = tf.get_variable("W4", shape=[6 * 4 * 128, 625],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([625]))
W5 = tf.get_variable("W5", shape=[625, nb_classes],initializer=tf.contrib.layers.xavier_initializer())    
b5 = tf.Variable(tf.random_normal([nb_classes]))

with tf.name_scope("layer1") as scope:    
    L1=tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding='SAME')
    L1=tf.nn.relu(L1)
    L1=tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    L1=tf.nn.dropout(L1,keep_prob=keep_prob)

with tf.name_scope("layer2") as scope:
    L2=tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding='SAME')
    L2=tf.nn.relu(L2)
    L2=tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    L2=tf.nn.dropout(L2,keep_prob=keep_prob)

with tf.name_scope("layer3") as scope:
    L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
    L3=tf.nn.relu(L3)
    L3=tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

    L3=tf.reshape(L3,[-1,6*4*128])
    L3=tf.nn.dropout(L3,keep_prob=keep_prob)

with tf.name_scope("layer4") as scope:
    L4=tf.nn.relu(tf.matmul(L3,W4)+b4)
    L4=tf.nn.dropout(L4,keep_prob=keep_prob)

with tf.name_scope("layer5") as scope:
    logits=tf.matmul(L4,W5)+b5

with tf.name_scope("cost") as scope:
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    cost_summ = tf.summary.scalar("cost", cost)
    





In [ ]:
learning_rate = 0.0005
with tf.name_scope("train") as scope:
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
with tf.Session() as sess:  
    merged_summary = tf.summary.merge_all()
    sess.run(tf.global_variables_initializer())
    writer=tf.summary.FileWriter('/home/jeongjaepark/HistNet/logs/maps')
    writer.add_graph(sess.graph)
    step = 0
    data.data_select([0,1,2,4,5,6,7,8,9])
# train my model
    print('Learning started. It takes sometime.')
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch =  data.train.total_batch

        for i in range(total_batch):
            batch_xs, batch_ys = data.train.Next_Batch()
            feed_dict = {X: batch_xs, Y: batch_ys,keep_prob:0.7}
            c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
            avg_cost += c / total_batch

            s=sess.run(merged_summary, feed_dict=feed_dict)
            writer.add_summary(s, global_step=step)
            step += 1
        
        
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    

In [1]:
7*32

224